<a href="https://colab.research.google.com/github/spoonia1/SAM_plot/blob/master/Task_1_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install transformers

     |████████████████████████████████| 1.3MB 8.7MB/s 
     |████████████████████████████████| 890kB 24.9MB/s 
     |████████████████████████████████| 1.1MB 40.6MB/s 
     |████████████████████████████████| 2.9MB 39.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=b95f2c749569b00292eb8b258a69faea398b016b1b1e6764f0ac10542dead11b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!nvidia-smi

Thu Nov 12 11:02:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

## **Import the dataset from github**

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Ankan-Das/Resume/master/Training-data-cleaned.tsv', sep='\t')

In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df.shape

(55513, 3)

In [ ]:
print(df[df['output']==1]['0'])[-1]

NameError: ignored

In [ ]:
df.shape[0]

55513

## **Loading SciBert Model**

In [ ]:
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_cased')

## **Loading the model in GPU**

In [ ]:
model = model.to('cuda')

## **The Loop**

In [ ]:
!pip install gputil
import GPUtil

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=e9f1159ca127f3f02bf7a1caa1ad84b7449f8f06f4afbcd68debd087bd4a3ec9
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [ ]:
# del CLS_val
# torch.cuda.empty_cache()
GPUtil.showUtilization()
# del CLS_val, last_hidden_states

| ID | GPU | MEM |
------------------
|  0 |  0% | 10% |


In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                            df:  9.7 MiB
ALL_PRETRAINED_CONFIG_ARCHIVE_MAP:  4.6 KiB
                CONFIG_MAPPING:  2.8 KiB
             TOKENIZER_MAPPING:  2.8 KiB
                 MODEL_MAPPING:  2.7 KiB
    MODEL_WITH_LM_HEAD_MAPPING:  2.5 KiB
 MODEL_FOR_PRETRAINING_MAPPING:  2.5 KiB
              PretrainedConfig:  2.0 KiB
                 BertTokenizer:  2.0 KiB
              TF_MODEL_MAPPING:  1.6 KiB


In [ ]:
# !pip install pympler

In [ ]:
# CLS_val[0].element_size()*CLS_val[0].nelement()*64000

In [ ]:
# del tokenizer
# del model

In [ ]:
# del last_hidden_states

In [ ]:
torch.manual_seed(271828)
np.random.seed(271728)

CLS_val = []

for i in range(111):

  print(i)

  batch_1 = df[i*500:(i+1)*500]           ## Batch Size of 500 sentences   Total: 55513


  ### Tokenization
  tokenized = batch_1['0'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))


  ### Padding
  max_len = 0
  for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

  ### Masking
  attention_mask = np.where(padded != 0, 1, 0)

  ### Making the input a tensor
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  ### Loading the inputs to GPU
  input_ids = input_ids.to('cuda')
  attention_mask = attention_mask.to('cuda')

  with torch.no_grad():
    last_hidden_states = list(model(input_ids))

  # print(type(last_hidden_states))
  ### Appending CLS to CLS_val
  CLS_val = CLS_val + list(last_hidden_states[0][:,0,:].cpu())

  #####################################################################
  ## UNLOAD inputs from GPU         ~~ Siddharth
  #####################################################################
  # input_ids = input_ids.detach()
  
  # print(last_hidden_states)
  # CLS_val = CLS_val.detach()
  torch.cuda.empty_cache()
  # print('end')
  # last_hidden_states = last_hidden_states.detach()
  del last_hidden_states, input_ids, attention_mask, tokenized


  #####################################################################


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110


In [ ]:
torch.cuda.empty_cache()

In [ ]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 10% |


In [ ]:
torch.cuda.empty_cache()
import gc
# del CLS_val, last_hidden_states, input_ids, attention_mask
gc.collect()


In [ ]:
# sys.getsizeof(CLS_val)

In [ ]:
type(CLS_val)

list

In [ ]:
features = np.array(CLS_val)

In [ ]:
type(features)

numpy.ndarray

In [ ]:
features.shape 

(55500, 768)

In [ ]:
labels = np.array(df['output'][:55500])

In [ ]:
labels.shape

(55500,)

## **Test Train Split**

In [ ]:
features.shape

(55500, 768)

In [ ]:
np.random.seed(3)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

## **Fitting in Logistic Regression**

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(test_features, test_labels)

0.9119279279279279

In [ ]:
from sklearn.metrics import f1_score
f1_score(test_labels, test_features, average='macro')

ValueError: ignored

In [ ]:
######################################################################################################
#########################           DUMMY CLASSIFIER               ###################################
######################################################################################################

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.835 (+/- 0.01)


## **SVM**

## **Gaussian NB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()


In [ ]:
gnb_clf = gnb.fit(train_features, train_labels)

test_predict_gnb = gnb_clf.predict(test_features)

In [ ]:
gnb_clf.score(test_features, test_labels)

0.31048648648648647

In [ ]:
confusion_matrix(test_labels, test_predict_gnb, normalize='true')

array([[0.2586833 , 0.7413167 ],
       [0.14581607, 0.85418393]])

## **Prediction**

We have to make an interface that takes in Input Sentences and predicts whether its important or not. 
We have to find the sentence embeddings as we did earlier, Only this time the logistic regression model will be a pretrained one, with an accuracy of 91.2%

In [ ]:
type(test_labels)

numpy.ndarray

In [ ]:
test_labels

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
test_predict = lr_clf.predict(test_features)

In [ ]:
test_predict

array([0., 0., 0., ..., 0., 0., 0.])

1. Should we do Over-Sampling? Or Under-Sampling?
2. 

## **Confusion Matrix**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

In [ ]:
confusion_matrix(test_labels, test_predict, normalize='true')

array([[0.99179034, 0.00820966],
       [0.92626346, 0.07373654]])

In [ ]:
plot_confusion_matrix(lr_clf, test_features, test_features)
plt.show()

ValueError: ignored

## **Interface**

In [ ]:
#### Take a group of Sentences as Input 


#### Do SciBert Embedding and take out the CLS values


#### Run the embeddings through the lr_clf model to get an array of outputs containing 0s and 1s


#### Extract the Sentences where it predicted 1

